In [ ]:
#multi-scale

In [ ]:
import numpy as np
import pandas as pd
import glob
import os
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from scipy.io import loadmat
import matplotlib.pylab as plt
import glob
from scipy.ndimage.interpolation import rotate
# import scipy
import sugartensor as tf
import sys
sys.path.append('../')
import UtilsLJ
from tqdm import tqdm
import sys

sys.path.append('./src/')
from model_multi import *
from utils import * 

#sys.path.append('./PointSetGeneration/depthestimate')

os.environ['CUDA_DEVICE_ORDER']="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']="0"

#About Data

batSize=2
maxStep=100000 # fixed with learningRate and learningRateDecay
learningRate=1e-2
decayStep = 30000
learningRateDecay=0.1
lrType=1 #learning rate type
lr2Period=5
adam_beta1=0.9 # check adam optimization
adam_beta2=0.99

maxGradient = 1

conWeightVar=['train/FeastNet','global_step']  #'FeastNet','global_step'# variables to be loaded
saveStep=80/batSize*10 # frequency to save weight
maxKeepWeights=2000 # how many records to save (for disk)
stepsContinue=-1  # from which steps continu.
#For Debug and results printing
keepProb=0.99999
# Totally dosen't work if put dropout after max-pool layer. 
printStep=80/batSize
ss=6890
dat="FEAST_FAUST-wen-LLE-Single-V6-18-Rerun"
Mconv = 32

lle_coef = 0.001
reg_coef = 1e-5
dirSave="./TFweights/Simple{}_Mconv_{}_Ss_{}_Bs_{}_lrT_{}_LR_{}_LRdecay_{}_Dkp_{}_Batchsize_{}_MaxGradient_{}_LLEcoef_{}_Regcoef{}".format(
    dat, Mconv, ss, batSize, lrType, learningRate, learningRateDecay,keepProb,batSize,maxGradient,lle_coef,reg_coef)

#conWeightPath="./TFweights/Simple2Dto3DMGPU_NR_1_FBN_0_Ss_1024_Bs_32_lrT_1_LR_0.001_LRdecay_0.999_Dkp_0.99999"
conWeightPath=""


RSW=8
RSD=8


In [ ]:
!nvidia-smi

In [ ]:
NUM_CLASSES = 6890
ARCHITECTURE = 0
# DATASET_PATH = FLAGS.dataset_path
# RESULTS_PATH = FLAGS.results_path
# NUM_ITERATIONS = FLAGS.num_iterations
# NUM_INPUT_CHANNELS = FLAGS.num_input_channels
# LEARNING_RATE = FLAGS.learning_rate
# NUM_POINTS = FLAGS.num_points
# NUM_CLASSES = FLAGS.num_classes


In [ ]:
def drawPlot(x,y,name):
    plt.rcParams['savefig.dpi'] = 300 
#    xmaxorLocator = MultipleLocator(1) 
#    plt.gca().xaxis.set_major_locator(xmaxorLocator)
#    plt.ylim(0,50)
    plt.plot(np.arange(0,len(x))*4,x,'k-',alpha=1,label='Train max: '+str(round(max(x),3))+', min: '+str(round(min(x),3)))
    plt.plot(np.arange(0,len(y)),y,'r-',alpha=1,label='Test max: '+str(round(max(y),3))+', min: '+str(round(min(y),3)))
    plt.legend()
    plt.xlabel('epoch',fontsize=9)
    plt.ylabel(name+' value',fontsize=9)
    plt.savefig(dirSave+"/"+name+".jpg",bbox_inches='tight')
    plt.show()

In [ ]:
x,y_,adj=UtilsLJ.read_from_tfrecords(
['./data/FAUST_FEAST_TRAIN_normilize.tfrecords'],['xyz','label','adj'],batSize,
                              [[6890,3],[6890],[6890,6]])

In [ ]:
x_val = tf.sg_print(x)
adj_val = tf.sg_print(adj)

In [ ]:
adj_val.min(),adj_val.max()

In [ ]:
def train():
    
    
    tf.reset_default_graph()
    tf.set_random_seed(888)
    print("*****************************************")
    print("Training started with random seed: {}".format(RSW))
    print("Batch started with random seed: {}".format(RSD))
    

    x,y_,adj=UtilsLJ.read_from_tfrecords(
    ['./data/FAUST_FEAST_TRAIN_normilize.tfrecords'],['xyz','label','adj'],batSize,
                                  [[6890,3],[6890],[6890,6]])
    

    x_t,y_t,adj_t=UtilsLJ.read_from_tfrecords(
    ['./data/FAUST_FEAST_TEST_normilize.tfrecords'],['xyz','label','adj'],batSize,
                                  [[6890,3],[6890],[6890,6]],shuffle=False)
    
    
    y_ = tf.cast(y_, dtype=tf.int64)
    y_t = tf.cast(y_t, dtype=tf.int64)
    adj = tf.cast(adj, dtype=tf.int32)
    adj_t = tf.cast(adj_t, dtype=tf.int32)
    
    global_step = tf.Variable(1, trainable=False,name='global_step')

    # get model and loss
    with tf.variable_scope("train") as scope1:
        y_conv,lle_loss = get_model_multi_lle(x, adj, NUM_CLASSES, ARCHITECTURE, Mconv)
        scope1.reuse_variables()
        y_conv_t,lle_loss_t = get_model_multi_lle(x_t, adj_t, NUM_CLASSES, ARCHITECTURE, Mconv)
    
    batch = tf.Variable(0, trainable=False)

    # Standard classification loss
    Loss = tf.reduce_mean(tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_, logits=y_conv), axis=1))
    Loss = Loss+ lle_loss * lle_coef
    reg_losses = sum(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))
    Loss = Loss+ reg_losses*reg_coef
    predictions = tf.argmax(y_conv, 2)
    correct_prediction = tf.equal(predictions, y_)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    Loss_t = tf.reduce_mean(tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_t, logits=y_conv_t), axis=1))
    predictions_t = tf.argmax(y_conv_t, 2)
    correct_prediction_t = tf.equal(tf.argmax(y_conv_t,2), y_t)
    accuracy_t = tf.reduce_mean(tf.cast(correct_prediction_t, tf.float32))
    
    print(Loss.shape, accuracy.shape, accuracy_t.shape)
    
    # Add summary writers
    #merged = tf.merge_all_summaries()
    
    
    
    #Learning Rate****************************************************************************
    
    lr = tf.train.exponential_decay(learningRate, global_step,
                                                  decayStep, learningRateDecay, staircase=True) 
    lr=tf.clip_by_value(lr, 1e-6,1)
    
    # Optimization Algo************************************************************************
    optimizer = tf.train.AdamOptimizer(learning_rate=lr,
                                                    beta1=adam_beta1,
                                                    beta2=adam_beta2
                                                   )
    gradients, variables = zip(*optimizer.compute_gradients(Loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
    train_step = optimizer.apply_gradients(zip(gradients, variables))
    
    saver = tf.train.Saver(max_to_keep=int(maxKeepWeights))
   
    init_op = tf.group(tf.global_variables_initializer(),
                               tf.local_variables_initializer())
    
    tf.summary.scalar('learning_rate', lr)
    tf.summary.scalar('loss_train', Loss)
    tf.summary.scalar('loss_train_lle', lle_loss)
    tf.summary.scalar('loss_train_reg', reg_losses)
#     tf.summary.scalar('loss_test', Loss_t)
    tf.summary.scalar('accuracy', accuracy)
#     tf.summary.scalar('accuracy_test', accuracy_t)

    merged = tf.summary.merge_all()
    
    # Continue Training************************************************************************
    if len(conWeightPath)>0:
        print("Continue Training...")
        tmp_var_list={}
        if len(conWeightVar)==0:
            print("For all variables")
            globals()['conWeightVar']={''}
        else:
            print("Training variables: {}".format(conWeightVar))
            
        for j in conWeightVar: 
            for i in tf.global_variables():
                if i.name.startswith(j):
                    tmp_var_list[i.name[:-2]] = i
                    
        saver1=tf.train.Saver(tmp_var_list) 
    # For Testing Dosent matther****************************************************************   
    
    
    # Training**********************************************************************************    
    with tf.Session() as sess:
        
        
        train_writer = tf.summary.FileWriter(os.path.join(dirSave, 'train'),
                                  sess.graph)
        test_writer = tf.summary.FileWriter(os.path.join(dirSave, 'test'))

        sess.run(init_op)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        # Read Weight******************************
        if len(conWeightPath)>0:
            print(conWeightPath)
            if stepsContinue==-1:            
                STEPS=sorted([int(i.split("/")[-1].split(".")[1].split("-")[-1]) for i in glob.glob(conWeightPath+"/*meta")])
                globals()['stepsContinue']=STEPS[-1]
                
            wtt=glob.glob(conWeightPath+"/*{}*meta".format(stepsContinue))[0][:-5]
            print("Reading Weight:{}".format(wtt))
            saver1.restore(sess,wtt)
            print('Weight is successfully updated from: {}'.format(wtt))  
        #*******************************************    
        
        stepst = sess.run(global_step)
        
        train_acc=[]
        test_acc=[]
        train_loss_list=[]
        test_loss_list=[]

        acc_epoch=[]
        acc_epoch_t=[]
        
        train_loss_epoch=0
        test_loss_epoch=0
        
        best_val_acc = 0
        bast_val_step = 0
        
        for t in tqdm(range(stepst,int(maxStep)+1)):
            
            step, summary, _, llr, Ltr, Ltr_lle, Ltr_reg = sess.run([global_step, merged, train_step, lr, Loss, lle_loss, reg_losses]) 

            
            if t==stepst:
                Ls=1
            else:
                Ls=0.99*Ls+0.01*Ltr
                
            #RL=0.999*RL+0.001*regL
            
            # Calculate Accuracy******************************
            train_loss, pred, train_y= sess.run([ Loss, y_conv, y_]) 
            train_correct_prediction = np.equal(np.argmax(pred,2), train_y)
            acc_epoch.append(train_correct_prediction)
            train_loss_epoch=train_loss_epoch + train_loss/(80/batSize)
            
            
            test_loss, pred_t, test_y= sess.run([ Loss_t,y_conv_t, y_t]) 
            test_correct_prediction = np.equal(np.argmax(pred_t,2), test_y)
            acc_epoch_t.append(test_correct_prediction)
            test_loss_epoch=test_loss_epoch + train_loss/(20/batSize)
            #*******************************************   
            
            # train epoch ******************************
            if t%(80/batSize)==0:            
                train_acc_epoch = np.mean(acc_epoch)
                train_acc.append(train_acc_epoch)
                acc_epoch=[]
                print('learning rate: %f, train loss: %f, train acc: %f, lle_loss: %f, reg_loss: %f' % (llr, train_loss_epoch, train_acc_epoch, Ltr_lle, Ltr_reg))
#                 print('gt_t: ', train_y, 'pred: ', np.argmax(pred,2))
                train_loss_list.append(train_loss_epoch)
                train_loss_epoch=0
                train_writer.add_summary(summary, step)
            #*******************************************   
            
            # test epoch ******************************
            if t%(20/batSize)==0:            
                test_acc_epoch = np.mean(acc_epoch_t)
                if test_acc_epoch>best_val_acc:
                    best_val_acc = test_acc_epoch
                    bast_val_step = t
                test_acc.append(test_acc_epoch)
                acc_epoch_t=[]
                print('learning rate: %f, test loss: %f, test acc: %f' % (llr, test_loss_epoch, test_acc_epoch))
#                 print('gt_t: ', test_y, 'pred_t: ', np.argmax(pred_t,2))
                test_loss_list.append(test_loss_epoch)
                test_loss_epoch=0
            #*******************************************  
                
#                 print('y_conv_val:', y_conv_val[0,:5,:])
            if t% 1000==0:
                if not os.path.exists(dirSave):
                    os.makedirs(dirSave)
                    
                with open(dirSave + '/result.txt', 'a') as f:
                    f.write(str(llr)+"*"+str(Ls)+"\n")
                    f.close()
                
                with open(dirSave + '/acc.txt', 'w') as f2:
                    f2.write(str(train_acc)+"\n"+str(test_acc)+"\n")
                    f2.close()

            if (t % saveStep==0):
                if not os.path.exists(dirSave):
                    os.makedirs(dirSave)
                saver.save(sess, dirSave + '/model.ckpt', global_step=t)
                
                
                if (len(train_loss_list)!=0) and (len(test_loss_list)!=0) and (len(train_acc)!=0) and (len(test_acc)!=0):
                    
                    drawPlot(train_loss_list,test_loss_list,"Loss")
                    drawPlot(train_acc,test_acc,"Accuracy")
        
        
        coord.request_stop()
        coord.join(threads)              

In [ ]:
train()

In [ ]:
# conWeightPath = './TFweights/SimpleFAUST-V2_NR_1_FBN_0_Ss_2048_Bs_8_lrT_1_LR_0.001_LRdecay_0.995_Dkp_0.99999/'

# stepsContinue = 240000

train()

In [ ]:
print(bast_val_step, best_val_acc)

In [ ]:
def test():
    tf.reset_default_graph()
    tf.set_random_seed(888)
    print("*****************************************")
    print("Training started with random seed: {}".format(RSW))
    print("Batch started with random seed: {}".format(RSD))
    

#     x,y_,adj=UtilsLJ.read_from_tfrecords(
#     ['./data/FAUST_FEAST_TRAIN_normilize.tfrecords'],['xyz','label','adj'],batSize,
#                                   [[6890,3],[6890],[6890,6]])
    

    x_t,y_t,adj_t=UtilsLJ.read_from_tfrecords(
    ['./data/FAUST_FEAST_TEST_normilize.tfrecords'],['xyz','label','adj'],batSize,
                                  [[6890,3],[6890],[6890,6]],shuffle=False)
    
    
#     y_ = tf.cast(y_, dtype=tf.int64)
    y_t = tf.cast(y_t, dtype=tf.int64)
#     adj = tf.cast(adj, dtype=tf.int32)
    adj_t = tf.cast(adj_t, dtype=tf.int32)
    
    global_step = tf.Variable(1, trainable=False,name='global_step')

    # get model and loss
    with tf.variable_scope("train") as scope1:
#         y_conv = get_model(x, adj, NUM_CLASSES, ARCHITECTURE, Mconv)
#         scope1.reuse_variables()
        y_conv_t,lle_loss_t = get_model_multi_lle(x_t, adj_t, NUM_CLASSES, ARCHITECTURE, Mconv)
    
    batch = tf.Variable(0, trainable=False)

#     Standard classification loss
#     Loss = tf.reduce_mean(tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_, logits=y_conv), axis=1))
#     predictions = tf.argmax(y_conv, 2)
#     correct_prediction = tf.equal(predictions, y_)
#     accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    Loss_t = tf.reduce_mean(tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_t, logits=y_conv_t), axis=1))
    predictions_t = tf.argmax(y_conv_t, 2)
    correct_prediction_t = tf.equal(tf.argmax(y_conv_t,2), y_t)
    accuracy_t = tf.reduce_mean(tf.cast(correct_prediction_t, tf.float32))
    
#     print(accuracy.shape, accuracy_t.shape)

    #Learning Rate****************************************************************************
    
#     lr = tf.train.exponential_decay(learningRate, global_step,
#                                                   decayStep, learningRateDecay, staircase=True) 
#     lr=tf.clip_by_value(lr, 1e-6,1)
    
#     # Optimization Algo************************************************************************
#     train_step = tf.train.AdamOptimizer(learning_rate=lr,
#                                                     beta1=adam_beta1,
#                                                     beta2=adam_beta2
#                                                    ).minimize(Loss,global_step=global_step)
    
#     saver = tf.train.Saver(max_to_keep=int(maxKeepWeights))
   
    init_op = tf.group(tf.global_variables_initializer(),
                               tf.local_variables_initializer())
    
    # Continue Training************************************************************************
    if len(conWeightPath)>0:
        print("Continue Training...")
        tmp_var_list={}
        if len(conWeightVar)==0:
            print("For all variables")
            globals()['conWeightVar']={''}
        else:
            print("Training variables: {}".format(conWeightVar))
            
        for j in conWeightVar: 
            for i in tf.global_variables():
                if i.name.startswith(j):
                    tmp_var_list[i.name[:-2]] = i
                    
        saver1=tf.train.Saver(tmp_var_list) 
    # For Testing Dosent matther****************************************************************   
    
    
    # Training**********************************************************************************    
    with tf.Session() as sess:
        sess.run(init_op)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        # Read Weight******************************
        if len(conWeightPath)>0:
            print(conWeightPath)
            if stepsContinue==-1:            
                STEPS=sorted([int(i.split("/")[-1].split(".")[1].split("-")[-1]) for i in glob.glob(conWeightPath+"/*meta")])
                globals()['stepsContinue']=STEPS[-1]
                
            wtt=glob.glob(conWeightPath+"/*{}*meta".format(stepsContinue))[0][:-5]
            print("Reading Weight:{}".format(wtt))
            saver1.restore(sess,wtt)
            print('Weight is successfully updated from: {}'.format(wtt))  
        #*******************************************    
        
        stepst = sess.run(global_step)
        
        cord_list = []
        pred_list = []
        gt_list = []
        acc_list = []
        prob_list = []
        for t in tqdm(range(20//batSize)):
            cord, pred_cur, gt_cur, acc_cur, y_conv_t_cur = sess.run([x_t, predictions_t,y_t,accuracy_t, y_conv_t])
            cord_list.append(cord)
            pred_list.append(pred_cur)
            gt_list.append(gt_cur)
            acc_list.append(acc_cur)
            prob_list.append(y_conv_t_cur)
        cord_list = np.concatenate(cord_list,axis=0)
        pred_list = np.concatenate(pred_list,axis=0)
        gt_list = np.concatenate(gt_list,axis=0)
        prob_list = np.concatenate(prob_list, axis=0)
        print('acc: %f' % (np.mean(acc_list)))
        
        coord.request_stop()
        coord.join(threads)
        
        return cord_list, pred_list, gt_list, acc_list, prob_list

In [ ]:
sorted(os.listdir(dirSave))

In [ ]:
stepsContinue=25200
conWeightPath=dirSave
cord_list, pred_list, gt_list, acc_list, prob_list = test()
print('acc: %f' % (np.mean(acc_list)))

In [ ]:
10000: 0.982, 16000:0.995, 18000:0.989, 25200: 0.97


In [ ]:
ls -l ./TFweights/SimpleFEAST_FAUST-wen-LLE-Single-V6-18_Mconv_32_Ss_6890_Bs_2_lrT_1_LR_0.01_LRdecay_0.1_Dkp_0.99999_Batchsize_2_MaxGradient_1_LLEcoef_0.001_Regcoef1e-05

In [ ]:
pred_list[0].max()

In [ ]:
ls data

In [ ]:
# os.mkdir('Result/')
np.save('Result/pred_V6-18-prob.npy',(pred_list,gt_list))

In [ ]:
#calculate mean distance error
dist_list = []
for s in range(cord_list.shape[0]):
    gt_xyz = cord_list[s][gt_list[s]]
    pred_xyz = cord_list[s][pred_list[s]]
    dist = np.sqrt(np.sum((gt_xyz-pred_xyz)**2, axis=1))
    dist = np.mean(dist)
    dist_list.append(dist)
dist_mean = np.mean(dist_list)
print('mean distance error: ', dist_mean)
np.save(os.path.join(dirSave,'dist_mean.npy'),dist_mean)

In [ ]:
#calculate accuracy under different distance threshod
#calculate mean distance error
acc_list = []
thr_list = np.arange(0,0.1,0.005)
for s in range(cord_list.shape[0]):
    for thr in thr_list:
        gt_xyz = cord_list[s][gt_list[s]]
        pred_xyz = cord_list[s][pred_list[s]]
        dist = np.sqrt(np.sum((gt_xyz-pred_xyz)**2, axis=1))
        acc = np.count_nonzero(dist<=thr)/float(ss)
        acc_list.append(acc)
acc_list = np.array(acc_list)
acc_list = acc_list.reshape((cord_list.shape[0], len(thr_list)))
acc_thr = np.mean(acc_list,axis=0)
print('mean accuracy under different distance threshod: ', acc_thr)
fig = plt.figure()
plt.plot(thr_list, acc_thr)
plt.xlim([0,0.101])
plt.ylim([0.9,1.01])
plt.xlabel('Geodesic error (% diameter)')
plt.ylabel('% correspondences')
plt.savefig(os.path.join(dirSave, 'acc.jpg'))
plt.show()

In [ ]:
import numpy as np
pred_list,gt_list = np.load('Result/pred_V6-18.npy')

In [ ]:
pred_list.shape,gt_list.shape